Create a column: 'GHG per EUR revenue_ranking_all_companies' - This is binned from 1 to 10 (using quartiles and calculated using only values 
from the same year)

Create a column: 'GHG per EUR revenue_ranking_sector' - This is also binned from 1 to 10 (and calculated using only values from the same year)

Calculate the average GHG per EUR revenue as well as IQR--apply the outlier transformation and put all outliers in the '0' bin

Create a column: 'GHG per EUR revenue_ranking_all_PY' -- This is to compare to the values from the previous year

Create a column: 'GHG per EUR revenue_ranking_sector_PY' -- This is to compare to the values from the previous year

Create a column: '% change in GHG per EUR revenue vs PY'

Create a column: '% change in GHG emissions vs PY'

Create a column: 'transport emissions as a % of total emissions'

Create a column: '% change in transport emissions vs PY'

Create a column: 'Transport emissions as % of total emissions' (compare to sector)


Use the bins only for GHG emissions/EUR--compare values in each bin for all columns...

See how bin values vary from year to year

Calculate the number of companies that have migrated from bin to bin





#### Bonus columns

'GHG intensity reduction % vs sector-specific targets'--normalise so make it a % above or below target

'GHG intensity reduction % vs others in the sector_CY'--also normalise (and consider whether positive is good or bad)




<center><span style="font-size:30px; font-weight: bold;">Nordic Compass Database</span></center>
<center><span style="font-size:24px;">Analysis of ESG Performance and CSRD Compliance</span></center>

<center><span style="font-size:22px;"><b>Section 3:</b> Impact analysis </span></center>

## Introduction to this section

## Imports

In [2]:
import pandas as pd
import numpy as np
import sys
import os

pd.set_option("display.max_columns", None)
sys.path.append(os.path.abspath(".."))
# import random

from functions import test_company, show_missing_values
from IPython.display import display

pd.options.display.float_format = "{:,.2f}".format

## Impact (materiality)

In [3]:
impact_df = pd.read_csv("../datasets/impact_df_original.csv")